# MHD matrix analysis
This code performs the cluster analysis to a distance matrix. It tries different combinations of parameters to find the best combination based on a mix of criteria.

We load the necessary packages.

In [ ]:
import hdbscan
import math
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import numpy as np
import os
from PIL import Image
from scipy.ndimage import sobel
from scipy.spatial.distance import cdist
from scipy.spatial import ConvexHull
from sklearn.manifold import TSNE
from sklearn.manifold import MDS
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, pairwise_distances_argmin_min
import shutil
import time
import torch
import torchvision.transforms as transforms
from umap import UMAP


Plotting the distance matrix.

In [ ]:
distance_matrix = np.load("Z:/data/test/blocks/final_distance_matrix.npy")

In [ ]:
# Plot the matrix
plt.magma()
plt.imshow(distance_matrix)
plt.title('MHD Matrix between all the pictures of the experiment', size = 10)
plt.colorbar(format='%.1f', label='Modified Hausdorff distance')
plt.show()

## General data overview 
### The images that least resemble each other

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import heapq

We will plot those pairs of photos that least resemble each other.  

We need to specify where are the original photos and the number of pairs to plot.

In [ ]:
# Parámetros de entrada
images_folder = 'Z:\data\exp20\deepest_all_every10'
num_pairs = 2  # Número de pares de imágenes a visualizar (cambiar según necesites)

In [ ]:
# Asegurarse de que la matriz es cuadrada (matriz de distancias simétrica)
assert distance_matrix.shape[0] == distance_matrix.shape[1], "La matriz de distancias no es cuadrada."

# Establecer las distancias de la diagonal principal a -1 para ignorarlas (ya que son 0, misma imagen)
np.fill_diagonal(distance_matrix, -1)

# Obtener todos los nombres de los archivos de imagen en la carpeta y ordenarlos alfabéticamente
image_files = sorted([f for f in os.listdir(images_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

# Asegurarse de que el número de imágenes coincide con la dimensión de la matriz de distancias
assert len(image_files) == distance_matrix.shape[0], "El número de imágenes no coincide con la dimensión de la matriz de distancias."

# Usar un heap para encontrar las mayores distancias
largest_distances = []  # Heap para las mayores distancias
for i in range(distance_matrix.shape[0]):
    for j in range(i + 1, distance_matrix.shape[1]):  # Solo recorremos la mitad superior de la matriz (matriz simétrica)
        distance = distance_matrix[i, j]
        if len(largest_distances) < num_pairs:
            heapq.heappush(largest_distances, (distance, i, j))
        else:
            heapq.heappushpop(largest_distances, (distance, i, j))

# Ordenar los pares de imágenes por la distancia en orden descendente
largest_distances.sort(reverse=True, key=lambda x: x[0])

# Plotear las imágenes de los pares más diferentes
fig, axes = plt.subplots(num_pairs, 2, figsize=(10, 5 * num_pairs))
for i, (dist_value, index_1, index_2) in enumerate(largest_distances):
    # Obtener los nombres de las imágenes basados en los índices
    image_1_name = image_files[index_1]
    image_2_name = image_files[index_2]
    
    # Construir las rutas completas de las imágenes
    image_1_path = os.path.join(images_folder, image_1_name)
    image_2_path = os.path.join(images_folder, image_2_name)
    
    # Cargar las imágenes
    image_1 = Image.open(image_1_path)
    image_2 = Image.open(image_2_path)
    
    # Mostrar las imágenes en el gráfico
    axes[i, 0].imshow(image_1)
    axes[i, 0].set_title(f'{image_1_name} (Índice {index_1})')
    axes[i, 0].axis('off')

    axes[i, 1].imshow(image_2)
    axes[i, 1].set_title(f'{image_2_name} (Índice {index_2})')
    axes[i, 1].axis('off')
    
    # Mostrar la distancia correspondiente
    print(f"Pares de imágenes: {image_1_name} (Índice {index_1}) y {image_2_name} (Índice {index_2}) - Distancia: {dist_value}")

plt.tight_layout()
plt.show()

## UMAP embedding + clustering with both HDBSCAN & DBSCAN

We will perform dimensionality reduction using UMAP and then apply clustering algorithms (HDBSCAN and DBSCAN) to the embedded data. The results will be saved in a structured directory as shown below:


```raw
Z:/data/test/results/
├── UMAP_nn10_md0.1_nc5/
│   ├── embedding.npy
│   ├── HDBSCAN_mcs10_msNone/
│   │   ├── cluster_labels.npy
│   │   ├── centroids/
│   │       ├── cluster_0_centroid.png
│   │       └── ...
│   ├── DBSCAN_eps0.5_ms15/
│   │   ├── cluster_labels.npy
│   │   ├── centroids/
│   │       └── ...
│   └── ...
└── ...
```

Specify the base directory for saving the results and the directory containing the original images


In [ ]:
# Directorio base para guardar los resultados
results_base_dir = "Z:/data/test/results-every10"
# Directorio donde se encuentran las imágenes originales
images_directory = "E:/Exp20/OPTdeepest_every10"

Define the parameter ranges for UMAP and the clustering algorithms

In [ ]:
# Parámetros ajustados
umap_n_neighbors = [10, 30, 50]
umap_min_dist = [0.0, 0.1, 0.3]
umap_n_components = [2, 5, 10, 20]  # Nuevas dimensiones

hdbscan_min_cluster_size = [10, 20, 50]
hdbscan_min_samples = [None, 5, 15]

dbscan_eps = [0.2, 0.5, 0.8]
dbscan_min_samples = [5, 15, 30]

In [2]:
# Import additional necessary packages

import os
import numpy as np
import umap
import hdbscan
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull
from PIL import Image
import itertools
import pandas as pd
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import calinski_harabasz_score



# Manejar los valores NaN en la matriz
max_distance = np.nanmax(distance_matrix[np.isfinite(distance_matrix)])
distance_matrix_filled = np.nan_to_num(distance_matrix, nan=max_distance)

# Índices de los puntos (para colorear según el tiempo)
indices = np.arange(distance_matrix.shape[0])

image_files = sorted([f for f in os.listdir(images_directory) if f.endswith('.png') or f.endswith('.jpg')])

# Crear combinaciones de parámetros
umap_params = list(itertools.product(umap_n_neighbors, umap_min_dist, umap_n_components))
hdbscan_params = list(itertools.product(hdbscan_min_cluster_size, hdbscan_min_samples))
dbscan_params = list(itertools.product(dbscan_eps, dbscan_min_samples))


# Asegurarse de que el directorio base existe
os.makedirs(results_base_dir, exist_ok=True)

# Lista para almacenar métricas de clustering
metrics_list = []

### UMAP Dimensionality Reduction
We iterate over all combinations of UMAP parameters to reduce the dimensionality of the data.


In [2]:
# Iterar sobre todas las combinaciones de parámetros de UMAP
for n_neighbors, min_dist, n_components in umap_params:
    print(f"Procesando UMAP con n_neighbors={n_neighbors}, min_dist={min_dist}, n_components={n_components}")
    # Realizar la proyección UMAP con los parámetros actuales
    reducer = umap.UMAP(
        n_components=n_components,
        metric='precomputed',
        random_state=42,
        min_dist=min_dist,
        n_neighbors=n_neighbors
    )
    embedding = reducer.fit_transform(distance_matrix_filled)
    
    # Normalizar el embedding para mejorar el clustering
    scaler = StandardScaler()
    embedding_scaled = scaler.fit_transform(embedding)
    
    # Crear directorio para esta combinación de parámetros UMAP
    umap_dir_name = f"UMAP_nn{n_neighbors}_md{min_dist}_nc{n_components}"
    umap_dir = os.path.join(results_base_dir, umap_dir_name)
    os.makedirs(umap_dir, exist_ok=True)
    
    # Guardar el embedding UMAP
    np.save(os.path.join(umap_dir, "embedding.npy"), embedding)
    
    # -----------------------
    # Clustering con HDBSCAN
    # -----------------------
    # We apply DBSCAN clustering on the UMAP-reduced data using different parameter combinations.
    
    for min_cluster_size, min_samples in hdbscan_params:
        print(f"  Procesando HDBSCAN con min_cluster_size={min_cluster_size}, min_samples={min_samples}")
        # Aplicar HDBSCAN con los parámetros actuales
        clusterer_hdbscan = hdbscan.HDBSCAN(
            min_cluster_size=min_cluster_size,
            min_samples=min_samples
        )
        cluster_labels_hdbscan = clusterer_hdbscan.fit_predict(embedding_scaled)
        
        # Calcular métricas de clustering
        num_clusters = len(set(cluster_labels_hdbscan)) - (1 if -1 in cluster_labels_hdbscan else 0)
        
        if num_clusters > 1:
            silhouette_avg = silhouette_score(embedding_scaled[cluster_labels_hdbscan != -1],
                                              cluster_labels_hdbscan[cluster_labels_hdbscan != -1])
        else:
            silhouette_avg = np.nan

        if num_clusters > 1:
            davies_bouldin = davies_bouldin_score(embedding_scaled[cluster_labels != -1], 
                                                  cluster_labels[cluster_labels != -1])
        else:
            davies_bouldin = np.nan

        if num_clusters > 1:
            calinski_harabasz = calinski_harabasz_score(embedding_scaled[cluster_labels != -1], 
                                                        cluster_labels[cluster_labels != -1])
        else:
            calinski_harabasz = np.nan

        
        
        
        # Guardar métricas en la lista
        metrics_list.append({
            'UMAP_n_neighbors': n_neighbors,
            'UMAP_min_dist': min_dist,
            'UMAP_n_components': n_components,
            'Algorithm': 'HDBSCAN',
            'Params': f"min_cluster_size={min_cluster_size}, min_samples={min_samples}",
            'Num_Clusters': num_clusters,
            'Silhouette_Score': silhouette_avg,
            'Davies_Bouldin_Index': davies_bouldin,
            'Calinski_Harabasz_Index': calinski_harabasz
        })
        
        # Crear directorio para esta combinación de parámetros HDBSCAN
        hdbscan_dir_name = f"HDBSCAN_mcs{min_cluster_size}_ms{min_samples}"
        hdbscan_dir = os.path.join(umap_dir, hdbscan_dir_name)
        os.makedirs(hdbscan_dir, exist_ok=True)
        
        # Guardar las etiquetas de cluster
        np.save(os.path.join(hdbscan_dir, "cluster_labels.npy"), cluster_labels_hdbscan)
        
        # Si n_components es 2, podemos generar gráficos
        if n_components == 2:
            # Visualizar y guardar el gráfico sin mostrarlo
            plt.figure(figsize=(12, 8))
            
            # Graficar los puntos de ruido por separado
            noise_mask_hdbscan = cluster_labels_hdbscan == -1
            plt.scatter(
                embedding[noise_mask_hdbscan, 0],
                embedding[noise_mask_hdbscan, 1],
                c='black',
                s=10,
                alpha=0.5,
                label='Ruido'
            )
            
            # Graficar los puntos asignados a clusters
            cluster_mask_hdbscan = ~noise_mask_hdbscan
            scatter_hdbscan = plt.scatter(
                embedding[cluster_mask_hdbscan, 0],
                embedding[cluster_mask_hdbscan, 1],
                c=indices[cluster_mask_hdbscan],
                cmap='viridis',
                s=30,
                alpha=0.8,
                label='Datos'
            )
            plt.colorbar(scatter_hdbscan, label='Índice (Tiempo)')
            
            # Generar y dibujar los polígonos para los clusters
            unique_clusters_hdbscan = np.unique(cluster_labels_hdbscan)
            unique_clusters_hdbscan = unique_clusters_hdbscan[unique_clusters_hdbscan != -1]  # Excluir el ruido
            
            # Configuración del polígono
            polygon_color = 'lightgray'
            polygon_alpha = 0.5
            
            for cluster_label in unique_clusters_hdbscan:
                # Obtener los puntos que pertenecen al cluster actual
                cluster_indices = np.where(cluster_labels_hdbscan == cluster_label)[0]
                cluster_points = embedding[cluster_indices]
                if cluster_points.shape[0] < 3:
                    continue  # No se puede calcular el ConvexHull con menos de 3 puntos
                # Calcular el ConvexHull
                hull = ConvexHull(cluster_points)
                # Crear el polígono
                polygon = Polygon(
                    cluster_points[hull.vertices],
                    facecolor=polygon_color,
                    edgecolor='none',
                    alpha=polygon_alpha
                )
                plt.gca().add_patch(polygon)
                # Calcular el centroide del cluster
                centroid = cluster_points.mean(axis=0)
                # Graficar el centroide con un punto rojo de tamaño 5
                plt.scatter(
                    centroid[0],
                    centroid[1],
                    c='red',
                    s=50,
                    marker='x',
                    label='Centroides' if cluster_label == unique_clusters_hdbscan[0] else ""
                )
            
            plt.title(f'UMAP(nn={n_neighbors}, md={min_dist}) + HDBSCAN(mcs={min_cluster_size}, ms={min_samples})')
            plt.xlabel('UMAP 1')
            plt.ylabel('UMAP 2')
            plt.legend()
            plt.tight_layout()
            # Guardar la figura
            plt.savefig(os.path.join(hdbscan_dir, "hdbscan_plot.png"))
            plt.close()
        
        # Guardar imágenes de centroides para HDBSCAN sin mostrar
        centroids_dir = os.path.join(hdbscan_dir, "centroids")
        os.makedirs(centroids_dir, exist_ok=True)
        for cluster_label in unique_clusters_hdbscan:
            cluster_indices = np.where(cluster_labels_hdbscan == cluster_label)[0]
            # En caso de que no haya puntos en el cluster
            if len(cluster_indices) == 0:
                continue
            cluster_points = embedding[cluster_indices]
            # Calcular el centroide del cluster en el espacio embebido
            centroid = cluster_points.mean(axis=0)
            # Calcular la distancia de cada punto del cluster al centroide
            distances_to_centroid = np.linalg.norm(cluster_points - centroid, axis=1)
            # Encontrar el índice del punto más cercano al centroide
            closest_point_idx = cluster_indices[np.argmin(distances_to_centroid)]
            # Obtener el nombre del archivo de imagen correspondiente
            image_filename = image_files[closest_point_idx]
            image_path = os.path.join(images_directory, image_filename)
            # Cargar y guardar la imagen
            image = Image.open(image_path)
            image.convert('L').save(os.path.join(centroids_dir, f"cluster_{cluster_label}_centroid.png"))
    
    # ----------------------
    # Clustering con DBSCAN
    # ----------------------
    # We apply DBSCAN clustering on the UMAP-reduced data using different parameter combinations.

    for eps_value, min_samples in dbscan_params:
        print(f"  Procesando DBSCAN con eps={eps_value}, min_samples={min_samples}")
        # Aplicar DBSCAN con los parámetros actuales
        dbscan_clusterer = DBSCAN(eps=eps_value, min_samples=min_samples)
        cluster_labels_dbscan = dbscan_clusterer.fit_predict(embedding_scaled)
        
        # Calcular métricas de clustering
        num_clusters = len(set(cluster_labels_dbscan)) - (1 if -1 in cluster_labels_dbscan else 0)
        if num_clusters > 1:
            silhouette_avg = silhouette_score(embedding_scaled[cluster_labels_dbscan != -1],
                                              cluster_labels_dbscan[cluster_labels_dbscan != -1])
        else:
            silhouette_avg = np.nan
        
        if num_clusters > 1:
            davies_bouldin = davies_bouldin_score(embedding_scaled[cluster_labels != -1], 
                                                  cluster_labels[cluster_labels != -1])
        else:
            davies_bouldin = np.nan

        if num_clusters > 1:
            calinski_harabasz = calinski_harabasz_score(embedding_scaled[cluster_labels != -1], 
                                                        cluster_labels[cluster_labels != -1])
        else:
            calinski_harabasz = np.nan
        
        
        # Guardar métricas en la lista
        metrics_list.append({
            'UMAP_n_neighbors': n_neighbors,
            'UMAP_min_dist': min_dist,
            'UMAP_n_components': n_components,
            'Algorithm': 'DBSCAN',
            'Params': f"eps={eps_value}, min_samples={min_samples}",
            'Num_Clusters': num_clusters,
            'Silhouette_Score': silhouette_avg,
            'Davies_Bouldin_Index': davies_bouldin,
            'Calinski_Harabasz_Index': calinski_harabasz
        })
        
        # Crear directorio para esta combinación de parámetros DBSCAN
        dbscan_dir_name = f"DBSCAN_eps{eps_value}_ms{min_samples}"
        dbscan_dir = os.path.join(umap_dir, dbscan_dir_name)
        os.makedirs(dbscan_dir, exist_ok=True)
        
        # Guardar las etiquetas de cluster
        np.save(os.path.join(dbscan_dir, "cluster_labels.npy"), cluster_labels_dbscan)
        
        # Si n_components es 2, podemos generar gráficos
        if n_components == 2:
            # Visualizar y guardar el gráfico sin mostrarlo
            plt.figure(figsize=(12, 8))
            
            # Graficar los puntos de ruido por separado
            noise_mask_dbscan = cluster_labels_dbscan == -1
            plt.scatter(
                embedding[noise_mask_dbscan, 0],
                embedding[noise_mask_dbscan, 1],
                c='black',
                s=10,
                alpha=0.5,
                label='Ruido'
            )
            
            # Graficar los puntos asignados a clusters
            cluster_mask_dbscan = ~noise_mask_dbscan
            scatter_dbscan = plt.scatter(
                embedding[cluster_mask_dbscan, 0],
                embedding[cluster_mask_dbscan, 1],
                c=indices[cluster_mask_dbscan],
                cmap='viridis',
                s=30,
                alpha=0.8,
                label='Datos'
            )
            plt.colorbar(scatter_dbscan, label='Índice (Tiempo)')
            
            # Generar y dibujar los polígonos para los clusters
            unique_clusters_dbscan = np.unique(cluster_labels_dbscan)
            unique_clusters_dbscan = unique_clusters_dbscan[unique_clusters_dbscan != -1]  # Excluir el ruido
            
            for cluster_label in unique_clusters_dbscan:
                # Obtener los puntos que pertenecen al cluster actual
                cluster_indices = np.where(cluster_labels_dbscan == cluster_label)[0]
                cluster_points = embedding[cluster_indices]
                if cluster_points.shape[0] < 3:
                    continue  # No se puede calcular el ConvexHull con menos de 3 puntos
                # Calcular el ConvexHull
                hull = ConvexHull(cluster_points)
                # Crear el polígono
                polygon = Polygon(
                    cluster_points[hull.vertices],
                    facecolor=polygon_color,
                    edgecolor='none',
                    alpha=polygon_alpha
                )
                plt.gca().add_patch(polygon)
                # Calcular el centroide del cluster
                centroid = cluster_points.mean(axis=0)
                # Graficar el centroide con un punto rojo de tamaño 5
                plt.scatter(
                    centroid[0],
                    centroid[1],
                    c='red',
                    s=50,
                    marker='x',
                    label='Centroides' if cluster_label == unique_clusters_dbscan[0] else ""
                )
            
            plt.title(f'UMAP(nn={n_neighbors}, md={min_dist}) + DBSCAN(eps={eps_value}, ms={min_samples})')
            plt.xlabel('UMAP 1')
            plt.ylabel('UMAP 2')
            plt.legend()
            plt.tight_layout()
            # Guardar la figura
            plt.savefig(os.path.join(dbscan_dir, "dbscan_plot.png"))
            plt.close()
        
        # Guardar imágenes de centroides para DBSCAN sin mostrar
        centroids_dir = os.path.join(dbscan_dir, "centroids")
        os.makedirs(centroids_dir, exist_ok=True)
        for cluster_label in unique_clusters_dbscan:
            cluster_indices = np.where(cluster_labels_dbscan == cluster_label)[0]
            if len(cluster_indices) == 0:
                continue
            cluster_points = embedding[cluster_indices]
            # Calcular el centroide del cluster
            centroid = cluster_points.mean(axis=0)
            # Calcular la distancia de cada punto del cluster al centroide
            distances_to_centroid = np.linalg.norm(cluster_points - centroid, axis=1)
            # Encontrar el índice del punto más cercano al centroide
            closest_point_idx = cluster_indices[np.argmin(distances_to_centroid)]
            # Obtener el nombre del archivo de imagen correspondiente
            image_filename = image_files[closest_point_idx]
            image_path = os.path.join(images_directory, image_filename)
            # Cargar y guardar la imagen
            image = Image.open(image_path)
            image.convert('L').save(os.path.join(centroids_dir, f"cluster_{cluster_label}_centroid.png"))

# Crear el DataFrame de métricas a partir de la lista
metrics_df = pd.DataFrame(metrics_list)

# Guardar las métricas de clustering en un archivo CSV
metrics_df.to_csv(os.path.join(results_base_dir, "clustering_metrics.csv"), index=False)

SyntaxError: invalid syntax (3954806918.py, line 1)